<a href="https://colab.research.google.com/github/vishal-burman/PyTorch-Architectures/blob/master/modeling_LeNet/test_sample_LeNet.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
! git clone https://github.com/vishal-burman/PyTorch-Architectures.git

In [2]:
%cd PyTorch-Architectures/modeling_LeNet/

/content/PyTorch-Architectures/modeling_LeNet


In [13]:
import time
import torch
import torch.nn.functional as F
from torch.utils.data import DataLoader
from torchvision import datasets, transforms
from model import LeNet

In [4]:
#############
# Settings
#############

# Hyperparameters
random_seed = 1
learning_rate = 0.001
batch_size = 128
num_epochs = 10

# Architecture
num_features = 32*32
num_classes = 10

# Other
device = torch.device("cuda:0" if torch.cuda.is_available() else "cpu")
grayscale = False

In [7]:
################
# CIFAR 10
################

train_dataset = datasets.CIFAR10(root = "data",
                                 train = True,
                                 transform = transforms.ToTensor(), # scales it to 0-1 range
                                 download = True
                                )

test_dataset = datasets.CIFAR10(root = "data",
                                train = False,
                                transform = transforms.ToTensor()
                               )

train_loader = DataLoader(dataset = train_dataset, batch_size = batch_size, num_workers = 8, shuffle = True)

test_loader = DataLoader(dataset = test_dataset, batch_size = batch_size, num_workers = 8, shuffle = False)

# Checking the dataset
for images, labels in train_loader:
    print("Image Dimensions: ", images.shape)
    print("Label Dimensions: ", labels.shape)
    break

Extracting data/cifar-10-python.tar.gz to data
Image Dimensions:  torch.Size([128, 3, 32, 32])
Label Dimensions:  torch.Size([128])


In [9]:
torch.manual_seed(random_seed)

model = LeNet(num_classes, grayscale)
model.to(device)

optimizer = torch.optim.Adam(model.parameters(), lr = learning_rate)

In [14]:
def compute_accuracy(model, data_loader, device):
    correct_pred, num_examples = 0, 0
    for i, (features, targets) in enumerate(data_loader):
        
        features = features.to(device)
        targets = targets.to(device)
        
        logits, probas = model(features)
        
        _, predicted_labels = torch.max(probas, 1)
        num_examples += targets.size(0)
        correct_pred += (predicted_labels == targets).sum()
    
    return correct_pred.float() / num_examples * 100

start_time = time.time()
for epoch in range(num_epochs):
    
    model.train()
    
    for i, (features, targets) in enumerate(train_loader):
        
        features = features.to(device)
        targets = targets.to(device)
        
        # Forward and Backprop
        logits, probas = model(features)
        cost = F.cross_entropy(logits, targets)
        optimizer.zero_grad()
        
        cost.backward()
        
        # Update model parameters
        optimizer.step()
        
        # LOGGING
        if not i % 50:
            print("Epoch: %03d/%03d || Batch: %03d/%03d || Cost: %.4f" % (epoch+1, num_epochs, i, len(train_loader), cost))
        
    model.eval()
    with torch.set_grad_enabled(False):
        print("Epoch: %03d/%03d || Train: %.3f%%" % (epoch+1, num_epochs, compute_accuracy(model, train_loader, device)))
    
    print("Time Elapsed: %.2f min" % ((time.time()-start_time)/60))

print("Total Training Time: %.2f min" % ((time.time()-start_time)/60))

Epoch: 001/010 || Batch: 000/391 || Cost: 2.2970
Epoch: 001/010 || Batch: 050/391 || Cost: 1.6929
Epoch: 001/010 || Batch: 100/391 || Cost: 1.6304
Epoch: 001/010 || Batch: 150/391 || Cost: 1.4826
Epoch: 001/010 || Batch: 200/391 || Cost: 1.4744
Epoch: 001/010 || Batch: 250/391 || Cost: 1.4785
Epoch: 001/010 || Batch: 300/391 || Cost: 1.4919
Epoch: 001/010 || Batch: 350/391 || Cost: 1.3508
Epoch: 001/010 || Train: 54.470%
Time Elapsed: 1.33 min
Epoch: 002/010 || Batch: 000/391 || Cost: 1.4622
Epoch: 002/010 || Batch: 050/391 || Cost: 1.2202
Epoch: 002/010 || Batch: 100/391 || Cost: 1.3271
Epoch: 002/010 || Batch: 150/391 || Cost: 1.1654
Epoch: 002/010 || Batch: 200/391 || Cost: 1.4047
Epoch: 002/010 || Batch: 250/391 || Cost: 1.2564
Epoch: 002/010 || Batch: 300/391 || Cost: 1.3212
Epoch: 002/010 || Batch: 350/391 || Cost: 1.2312
Epoch: 002/010 || Train: 59.430%
Time Elapsed: 2.67 min
Epoch: 003/010 || Batch: 000/391 || Cost: 1.2066
Epoch: 003/010 || Batch: 050/391 || Cost: 1.1034
Epoch: